This is my code of the parsrer for news extraction from Chronicling America website. It allows to download news from the site and saves them in the .csv format. You can see an example of it's working process and results below on a small batch of data.

In [ ]:
!pip install fake_useragent

In [2]:
from bs4 import BeautifulSoup
import requests
import random
from fake_useragent import UserAgent
from tqdm.auto import tqdm
import pandas as pd
import random
import json
import time
from datetime import datetime
ua = UserAgent()
session = requests.session()

This function extracts information about date, when the newspaper was published, title, text and state, where the newspaper was published.

In [3]:
def get_page(page_number):
  r = session.get(f"https://chroniclingamerica.loc.gov/search/pages/results/?date1=1904&rows=20&searchType=basic&state=&date2=1905&proxtext=Japan&y=18&x=9&dateFilterType=yearRange&page={page_number}&sort=relevance&format=json", headers={'User-Agent': ua.random}).json()
  time.sleep(random.uniform(8.4, 9.4))
  try:
    items = r['itemsPerPage']
    result = []
    for i in range(items):
      block = {}
      try:
        obj = r['items'][i]
        if obj['language'][0] == 'English':
          try:
            block['year'] = obj['date'][0:4]
            block['month'] = obj['date'][4:6]
            block['day'] = obj['date'][6:8]
            block['title'] = obj['title']
            block['state'] = obj['state'][0]
            block['text'] = obj['ocr_eng']
            result.append(block)
          except Exception as e:
            print(e)
      except Exception as e:
          print(e)
  except Exception as e:
        print(e)
  return result

The processed data is then saved in a dataframe.

In [4]:
blocks = []
for i in tqdm(range(5)):
  blocks.extend(get_page(i+1))
df_final = pd.DataFrame(blocks).drop_duplicates()

  0%|          | 0/5 [00:00<?, ?it/s]

list index out of range


In [5]:
df_final.head()

,year,month,day,title,state,text
0,1904,01,10,New-York tribune. [volume],New York,"ILLUSTRATED SUPPLEMENT.\nSCENES IN COREA, THE ..."
1,1904,02,27,Virginia gazette. [volume],Virginia,"cnj S,-""'"":\n.\ni\n.\n*\nXXI!\n?\n.\n*\n??*?'?..."
2,1904,05,22,New-York tribune. [volume],New York,ILLUSTRATED SUPPLEMENT.\nA VIEW OF THE HARBOR ...
3,1904,07,31,New-York tribune. [volume],New York,SOME OF THE MOST PICTURESQUE THINGS TO BE SEEN...
4,1904,01,17,New-York tribune. [volume],New York,ILLUSTRATED SUPPLEMENT.\nWAR EXPERTS OF THE WO...


The following code allows to filter extracted news depending on their date of publishment. Such system allows to evaluate changes in data through time  more precisely. Data for each month of each year is saved in .json format afterwards.

In [13]:
data_1904 = {1: [], 2: [], 3: [], 4: [],
        5: [], 6: [], 7: [], 8: [],
        9: [], 10: [], 11: [], 12: []}

data_1905 = {1: [], 2: [], 3: [], 4: [],
        5: [], 6: [], 7: [], 8: [],
        9: [], 10: [], 11: [], 12: []}

In [14]:
for i in tqdm(range(len(df_final))):
  year = int(df_final['year'][i])
  month = int(df_final['month'][i])
  title = df_final['title'][i]
  text = df_final['text'][i]
  state = df_final['state'][i]
  if year == 1904:
    data_1904[month].append([title, text, state])
  else:
    data_1905[month].append([title, text, state])

  0%|          | 0/98 [00:00<?, ?it/s]

A data sample from the data collection. This is a list, that contains a name of a newspaper, text and a state, where the newspaper was published.

In [15]:
data_1904[1][1]

['New-York tribune. [volume]',
 "ILLUSTRATED SUPPLEMENT.\nWAR EXPERTS OF THE WORLD ARE NOW COMPARING THE FIGHTING FORCES OF RUSSIA AND JAPAN.\nBUSS IAN SOLDIERS IN CAMP.\nRUSSIAN SOLDIERS ON THE MARCH.\nGUNNERS OF THE RUSSIAN WARSHIP RETVIZAN.\nSUNDAY. JAJtfUAHY 17. 1904.\nRUSSIAN SOLDIERS BOARDING A TRAIN.\nJAPANESE ARMORED CIU'ISER YAKUMQ.\nA RUSSIAN WARSHIP.",
 'New York']